Sample BI queries
- Trend of daily public net electricity production in Germany for each production type. 
- Prediction of underperformance of public net electricity on 30min intervals.
- Analysis of daily price against the net power for offshore and onshore wind (= production_type)

In [2]:
import os
from dotenv import load_dotenv
from pyspark.sql import SparkSession
from pyspark.sql.functions import trunc,avg

In [3]:
# Get Jar path needed for spark session
# For simplicity using locally downloaded jars for delta format
cwd = os.getcwd()
if cwd.endswith("notebooks"):
    proj_dir = os.path.abspath("..")
else:
    proj_dir = cwd
jar_dir = os.path.join(proj_dir, "jars")
jar1 = os.path.join(jar_dir, "delta-spark_2.13-4.0.0.jar")
jar2 = os.path.join(jar_dir, "delta-storage-4.0.0.jar")

In [4]:
spark = SparkSession.builder.appName("EnergyBI_Insights") \
            .config("spark.jars", f"{jar1},{jar2}") \
            .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
            .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
            .config("spark.sql.warehouse.dir", f"{proj_dir}/data-warehouse") \
            .getOrCreate()


25/07/01 21:14:41 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [5]:
spark.conf.get("spark.sql.warehouse.dir")

'file:/Users/zodenath/Desktop/LOCAL/energy-proj/data-warehouse'

## Loading data to SQL warehouse

In [7]:
spark.sql(""" CREATE SCHEMA IF NOT EXISTS energy """)

DataFrame[]

In [8]:
data_lake_path = f"{proj_dir}/data/silver/public_power_data"

spark.sql(f"""
            CREATE TABLE IF NOT EXISTS energy.public_power_data
            USING DELTA
            LOCATION '{data_lake_path}'
""")

price_lake_path = f"{proj_dir}/data/silver/price_data"

spark.sql(f"""
            CREATE TABLE IF NOT EXISTS energy.price
            USING DELTA
            LOCATION '{price_lake_path}'
""")

installed_lake_path = f"{proj_dir}/data/silver/public_power_data"

spark.sql(f"""
            CREATE TABLE IF NOT EXISTS energy.installed_power_data
            USING DELTA
            LOCATION '{installed_lake_path}'
""")



DataFrame[]

In [9]:
# Trend of daily public net electricity production in Germany for each production type.

daily_trend = spark.sql("""
    SELECT
        DATE(timestamp) as date,
        production_type,
        SUM(net_power_produced) AS daily_net_production
    FROM energy.public_power_data
    GROUP BY DATE(timestamp), production_type
    ORDER BY date, production_type
""")
daily_trend.show(10)

25/07/01 21:15:28 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+----------+--------------------+--------------------+
|      date|     production_type|daily_net_production|
+----------+--------------------+--------------------+
|2025-06-30|             Biomass|   76155.49926757812|
|2025-06-30|Cross border elec...|  145359.10009765625|
|2025-06-30|Fossil brown coal...|            185681.0|
|2025-06-30|Fossil coal-deriv...|             10088.0|
|2025-06-30|          Fossil gas|            183015.0|
|2025-06-30|    Fossil hard coal|             81534.0|
|2025-06-30|          Fossil oil|              5394.0|
|2025-06-30|          Geothermal|   286.9000072479248|
|2025-06-30|  Hydro Run-of-River|   37398.59997558594|
|2025-06-30|Hydro pumped storage|             79601.0|
+----------+--------------------+--------------------+
only showing top 10 rows


In [23]:
# Prediction of underperformance of public net electricity on 30min intervals.
unperform_prediction = spark.sql("""
    SELECT
        production_type,
        window.start AS interval_start,
        window.end AS interval_end,
        SUM(net_power_produced) AS total_power
    FROM (
        SELECT *, window(timestamp, '30 minutes') AS window
        FROM energy.public_power_data
    )
    GROUP BY production_type, window
    ORDER BY interval_start
""")
unperform_prediction.show(10)


+--------------------+-------------------+-------------------+------------------+
|     production_type|     interval_start|       interval_end|       total_power|
+--------------------+-------------------+-------------------+------------------+
|Fossil brown coal...|2025-04-01 00:00:00|2025-04-01 00:30:00|  23208.2998046875|
|               Solar|2025-04-01 00:00:00|2025-04-01 00:30:00|               0.0|
|Cross border elec...|2025-04-01 00:00:00|2025-04-01 00:30:00|   9019.2001953125|
|       Residual load|2025-04-01 00:00:00|2025-04-01 00:30:00|     79468.8984375|
|       Wind offshore|2025-04-01 00:00:00|2025-04-01 00:30:00|1679.7999877929688|
|        Wind onshore|2025-04-01 00:00:00|2025-04-01 00:30:00|  13683.7001953125|
|Hydro pumped storage|2025-04-01 00:00:00|2025-04-01 00:30:00| 659.3000183105469|
|          Fossil gas|2025-04-01 00:00:00|2025-04-01 00:30:00|  15752.2001953125|
|               Waste|2025-04-01 00:00:00|2025-04-01 00:30:00|            1781.5|
|Fossil coal-der

In [24]:
# Analysis of daily price against the net power for offshore and onshore wind (= production_type)

price_vs_power = spark.sql("""
    SELECT
        DATE(p.timestamp) AS date,
        p.production_type,
        SUM(p.net_power_produced) AS total_power,
        AVG(pr.price) AS avg_price
    FROM energy.public_power_data p
    JOIN energy.price pr
        ON p.timestamp = pr.timestamp
    WHERE p.production_type IN ('Wind offshore', 'Wind onshore')
    GROUP BY DATE(p.timestamp), p.production_type
    ORDER BY date
""")
price_vs_power.show(10)

+----------+---------------+-----------------+-----------------+
|      date|production_type|      total_power|        avg_price|
+----------+---------------+-----------------+-----------------+
|2025-06-28|  Wind offshore|92742.40063476562|68.32333257463243|
|2025-06-28|   Wind onshore| 221999.998046875|68.32333257463243|
|2025-06-29|  Wind offshore|94731.20101928711|42.32374995946884|
|2025-06-29|   Wind onshore|         300561.0|42.32374995946884|
+----------+---------------+-----------------+-----------------+



In [ ]:
# Example Business query : Get monthly avr price and installed capacity
# Example of case where we read data directly from silver layer and get business insights
price = spark.read.format("delta").load(f"{proj_dir}/data/silver/price_data")  
installed_power = spark.read.format("delta").load(f"{proj_dir}/data/silver/installed_power_data")   

price = price.withColumn("month_start", trunc("timestamp", "month"))

price_monthly = price.withColumn("month", trunc("timestamp", "month")) \
                          .groupBy("month") \
                          .agg(avg("price").alias("avg_price"))
installed_monthly = installed_power.groupBy("date", "production_type") \
                                   .agg(avg("installed_power").alias("avg_capacity"))

joined_df = price_monthly.join(
    installed_monthly,
    price_monthly["month"] == installed_monthly["date"]
).drop("date")

In [11]:
joined_df.show()

+----------+-----------------+--------------------+------------------+
|     month|        avg_price|     production_type|      avg_capacity|
+----------+-----------------+--------------------+------------------+
|2025-06-01|200.4925022125244|Battery Storage (...|20.910999298095703|
|2025-06-01|200.4925022125244|         Solar gross|107.09200286865234|
|2025-06-01|200.4925022125244|        Wind onshore|              NULL|
|2025-06-01|200.4925022125244|Battery Storage (...|14.178000450134277|
|2025-06-01|200.4925022125244|             Biomass|              NULL|
|2025-06-01|200.4925022125244|           Solar net| 95.50800323486328|
|2025-06-01|200.4925022125244|       Wind offshore|              NULL|
+----------+-----------------+--------------------+------------------+



## Ad-hoc analytics without creating tables

In [33]:
#public_power = spark.read.format("delta").load(f"{proj_dir}/data/silver/public_power_data")  
#price = spark.read.format("delta").load(f"{proj_dir}/data/silver/price_data")   
#installed_power = spark.read.format("delta").load(f"{proj_dir}/data/silver/installed_power_data")  

In [34]:
#public_power.createOrReplaceTempView("net_power")
#price.createOrReplaceTempView("price")
#installed_power.createOrReplaceTempView("installed_power")
